In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import timedelta
from darts import TimeSeries
from darts.models import NBEATSModel
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
import torch

# ------------------------------------------------------------------------------
# 1. SETUP & PARAMETERS
# ------------------------------------------------------------------------------
START_DATE = pd.Timestamp("2011-01-29")
HORIZON = 28

# N-BEATS hyperparameters
input_chunk_length  = 28
output_chunk_length = 14
n_epochs            = 10

print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device Name: {torch.cuda.get_device_name(0)}")

# ------------------------------------------------------------------------------
# 2. DATA LOADING & MERGING
# ------------------------------------------------------------------------------
print("Loading data...")
# Load Train (d_1 - d_1913)
train_df = pd.read_csv("sales_train_validation_afcs2025.csv")

# Load Validation (d_1914 - d_1941)
valid_df = pd.read_csv("sales_test_validation_afcs2025.csv")

# Extract 'd_' columns and ensure they are sorted numerically
def get_d_cols(df):
    cols = [c for c in df.columns if c.startswith("d_")]
    return sorted(cols, key=lambda x: int(x.split('_')[1]))

train_cols = get_d_cols(train_df)
valid_cols = get_d_cols(valid_df)

# Align Validation IDs to Training IDs (just in case)
ids = train_df['id'].values
valid_df_aligned = valid_df.set_index('id').reindex(ids).reset_index()

# Merge the columns to create one wide DataFrame (d_1 ... d_1941)
# We take static cols from train_df and append the validation d_cols
static_cols = [c for c in train_df.columns if c not in train_cols]
sales_wide = pd.concat([train_df[static_cols + train_cols], valid_df_aligned[valid_cols]], axis=1)

print("Reshaping data (Melt)... this may take a moment.")
# All d_ columns in the new combined dataframe
all_d_cols = train_cols + valid_cols

sales_long = sales_wide.melt(
    id_vars=static_cols,
    value_vars=all_d_cols,
    var_name='d',
    value_name='y'
)

# Convert 'd_xx' -> datetime
print("Converting dates...")
sales_long['day_num'] = sales_long['d'].str.slice(2).astype(int)
sales_long['ds'] = START_DATE + pd.to_timedelta(sales_long['day_num'] - 1, unit='D')

# Final cleanup for Darts
sales_df = (
    sales_long[['id', 'ds', 'y']]
    .rename(columns={'id': 'unique_id'})
    .sort_values(by=['unique_id', 'ds'])
    .reset_index(drop=True)
)

# Ensure proper dtypes
sales_df["unique_id"] = sales_df["unique_id"].astype(str)
sales_df["y"] = pd.to_numeric(sales_df["y"], errors="coerce")

# ------------------------------------------------------------------------------
# 3. DARTS TIMESERIES CREATION
# ------------------------------------------------------------------------------
print("Building TimeSeries objects...")
series_list = TimeSeries.from_group_dataframe(
    sales_df,
    group_cols="unique_id",
    time_col="ds",
    value_cols="y",
    fill_missing_dates=True,
    freq='D'
)

# ------------------------------------------------------------------------------
# 4. COVARIATES GENERATION
# ------------------------------------------------------------------------------
print("Generating covariates...")

# We need covariates for History + Forecast Horizon
min_date = sales_df['ds'].min()
max_date = sales_df['ds'].max()
forecast_end_date = max_date + timedelta(days=HORIZON)

# Create global timeline
full_time_index = pd.date_range(start=min_date, end=forecast_end_date, freq='D')

# Dummy series for attributes
global_cov_ts = TimeSeries.from_times_and_values(
    times=full_time_index,
    values=np.zeros(len(full_time_index))
)

# Generate Calendar Features
month_cov = datetime_attribute_timeseries(global_cov_ts, attribute="month", one_hot=True)
day_cov = datetime_attribute_timeseries(global_cov_ts, attribute="dayofweek", one_hot=True)
global_covariates = month_cov.stack(day_cov)

# Replicate for each series (Darts expects list of covariates matching list of series)
covariates_list = [global_covariates] * len(series_list)

# Scale targets
print("Scaling targets...")
scaler = Scaler()
series_list_scaled = scaler.fit_transform(series_list)

# ------------------------------------------------------------------------------
# 5. MODEL TRAINING
# ------------------------------------------------------------------------------
print("Initializing N-BEATS...")
model = NBEATSModel(
    input_chunk_length=int(input_chunk_length),
    output_chunk_length=int(output_chunk_length),
    n_epochs=int(n_epochs),
    random_state=42,
    model_name="nbeats-full-history",
    force_reset=True,
    save_checkpoints=False,
    pl_trainer_kwargs={
        "accelerator": "gpu",
        "devices": 1
    },
    batch_size=1024
)

print("Fitting model on full history...")
model.fit(
    series=series_list_scaled,
    past_covariates=covariates_list,
    verbose=True
)

# ------------------------------------------------------------------------------
# 6. FORECASTING & EXPORT
# ------------------------------------------------------------------------------
print(f"Predicting next {HORIZON} days...")
preds_scaled = model.predict(
    n=HORIZON,
    series=series_list_scaled,
    past_covariates=covariates_list
)

print("Inverse scaling...")
preds = scaler.inverse_transform(preds_scaled)

print("Formatting submission...")
# Convert Darts TimeSeries objects back to DataFrame format
# preds is a list of TimeSeries. We need to stack them.
pred_rows = []
for i, ts in enumerate(preds):
    # Extract ID from the static info or original list order
    s_id = series_list[i].static_covariates_values()[0,0] if series_list[i].static_covariates is not None else ids[i]

    # Get values (Shape: Horizon x 1)
    vals = ts.values().flatten()

    # Create row: [id, F1, F2, ... F28]
    row = [s_id] + list(vals)
    pred_rows.append(row)

# Define columns
f_cols = [f"F{x}" for x in range(1, HORIZON + 1)]
submission_df = pd.DataFrame(pred_rows, columns=['id'] + f_cols)

# Save
submission_file = "submission_nbeats_test.csv"
submission_df.to_csv(submission_file, index=False)
print(f"Saved submission to {submission_file}")

CUDA Available: True
Device Name: NVIDIA A100-SXM4-40GB
Loading data...
Reshaping data (Melt)... this may take a moment.
Converting dates...
Building TimeSeries objects...
Generating covariates...
Scaling targets...


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Initializing N-BEATS...
Fitting model on full history...


INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 10.5 M | train
-------------------------------------------------------------
10.5 M    Trainable params
4.6 K     Non-trainable params
10.5 M    Total params
41.832    Total estimated model params size (MB)
396       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting next 28 days...


Predicting: |          | 0/? [00:00<?, ?it/s]

Inverse scaling...
Formatting submission...
Saved submission to submission_nbeats_test.csv


In [ ]:

# ------------------------------------------------------------------------------
# 6. FORECASTING & SUBMISSION GENERATION
# ------------------------------------------------------------------------------
print("Predicting...")
forecasts_scaled = model.predict(
    n=horizon,
    series=series_list_scaled,
    past_covariates=covariates_list,
    verbose=True
)

# Inverse scale
forecasts = scaler.inverse_transform(forecasts_scaled)

print("Formatting submission...")
# Convert list of TimeSeries back to a DataFrame
fcst_dfs = []
for ts, fc in zip(series_list, forecasts):
    # Extract the ID from the static covariates or components
    # Note: from_group_dataframe puts the group ID in static_covariates
    s_id = ts.static_covariates.iloc[0,0] if ts.static_covariates is not None else "unknown"

    # Create DataFrame: ds, yhat
    df_fc = fc.to_dataframe().reset_index()
    # Column 0 is time, Column 1 is the prediction
    df_fc.columns = ['ds', 'yhat']
    df_fc['id'] = s_id
    fcst_dfs.append(df_fc)

forecast_long = pd.concat(fcst_dfs, ignore_index=True)

# Pivot to Wide Format (id, F1...F28)
# We need to rank the dates to get F1, F2, etc.
forecast_long['F_col'] = forecast_long.groupby('id')['ds'].rank(method='first').astype(int)
forecast_long['F_col'] = "F" + forecast_long['F_col'].astype(str)

submission_df = forecast_long.pivot(index='id', columns='F_col', values='yhat').reset_index()

# Ensure we match the sample submission order and columns
sample_sub = pd.read_csv("sample_submission_afcs2025.csv")
# Right join or reindex to ensure we have all IDs in correct order
final_submission = sample_sub[['id']].merge(submission_df, on='id', how='left')

# Fill NaNs with 0 if any
final_submission = final_submission.fillna(0)

# Save
output_path = "submission_nbeats.csv"
final_submission.to_csv(output_path, index=False)
print(f"Submission saved to {output_path}")
print(final_submission.head())

Predicting...


AttributeError: 'NoneType' object has no attribute 'set_predict_parameters'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt

# 1. Load the Data
# Ground Truth (Validation set)
gt_df = pd.read_csv("sales_test_validation_afcs2025.csv")

# Predictions (Your generated submission file)
pred_df = pd.read_csv("submission_nbeats.csv")

# 2. Align DataFrames by ID
# We verify that we are comparing the same items in the same order.
# Sort both by 'id' to ensure row-to-row correspondence.
gt_df = gt_df.sort_values("id").reset_index(drop=True)
pred_df = pred_df.sort_values("id").reset_index(drop=True)

# Verify IDs match
if not gt_df["id"].equals(pred_df["id"]):
    print("Warning: IDs do not match perfectly. Performing an inner join alignment...")
    # Rename columns to avoid collisions if necessary, or just merge on ID
    # Here we simply filter to common IDs
    common_ids = pd.Series(list(set(gt_df['id']) & set(pred_df['id'])))
    gt_df = gt_df[gt_df['id'].isin(common_ids)].sort_values('id').reset_index(drop=True)
    pred_df = pred_df[pred_df['id'].isin(common_ids)].sort_values('id').reset_index(drop=True)

# 3. Extract Values
# Ground Truth columns: d_1914, d_1915, ..., d_1941 (excluding 'id')
y_true = gt_df.drop(columns=["id"]).values

# Prediction columns: F1, F2, ..., F28 (excluding 'id')
y_pred = pred_df.drop(columns=["id"]).values

# 4. Calculate RMSE
# Flatten arrays to compute global RMSE across all series and time steps
rmse = sqrt(mean_squared_error(y_true, y_pred))

print(f"Validation RMSE: {rmse:.5f}")

Validation RMSE: 3.17983


In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import timedelta
from darts import TimeSeries
from darts.models import NBEATSModel
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
from sklearn.metrics import mean_squared_error
from math import sqrt
import torch

# ------------------------------------------------------------------------------
# 1. SETUP & PARAMETERS
# ------------------------------------------------------------------------------
START_DATE = pd.Timestamp("2011-01-29")
HORIZON = 28

# N-BEATS hyperparameters
input_chunk_length  = 28
output_chunk_length = 14
n_epochs            = 10

print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device Name: {torch.cuda.get_device_name(0)}")

# ------------------------------------------------------------------------------
# 2. DATA LOADING & MERGING (TRAIN + VALIDATION)
# ------------------------------------------------------------------------------
print("Loading data...")
# 1. Load Train (d_1 - d_1913)
train_df = pd.read_csv("sales_train_validation_afcs2025.csv")

# 2. Load Validation (d_1914 - d_1941)
valid_df = pd.read_csv("sales_test_validation_afcs2025.csv")

# 3. Load Evaluation (Ground Truth for d_1942 - d_1969)
eval_df = pd.read_csv("sales_test_evaluation_afcs_2025.csv")

# Helper to sort d_ cols
def get_d_cols(df):
    cols = [c for c in df.columns if c.startswith("d_")]
    return sorted(cols, key=lambda x: int(x.split('_')[1]))

train_cols = get_d_cols(train_df)
valid_cols = get_d_cols(valid_df)

print(f"Training cols: {len(train_cols)}, Validation cols: {len(valid_cols)}")

# Align IDs (Ensure rows match)
ids = train_df['id'].values
valid_df = valid_df.set_index('id').reindex(ids).reset_index()
eval_df = eval_df.set_index('id').reindex(ids).reset_index()

# Combine Train + Validation into one DataFrame for training
# We take static cols from train_df and append the validation d_cols
static_cols = [c for c in train_df.columns if c not in train_cols]
# Ensure we only have static cols + d_cols (filtering out any accidental extras)
full_train_wide = pd.concat([train_df[static_cols + train_cols], valid_df[valid_cols]], axis=1)

print("Reshaping training data (Melt)... this may take a moment.")
all_train_cols = train_cols + valid_cols # d_1 to d_1941

sales_long = full_train_wide.melt(
    id_vars=static_cols,
    value_vars=all_train_cols,
    var_name='d',
    value_name='y'
)

# Convert 'd_xx' -> datetime
print("Converting dates...")
sales_long['day_num'] = sales_long['d'].str.slice(2).astype(int)
sales_long['ds'] = START_DATE + pd.to_timedelta(sales_long['day_num'] - 1, unit='D')

# Final cleanup for Darts
sales_df = (
    sales_long[['id', 'ds', 'y']]
    .rename(columns={'id': 'unique_id'})
    .sort_values(by=['unique_id', 'ds'])
    .reset_index(drop=True)
)

sales_df["unique_id"] = sales_df["unique_id"].astype(str)
sales_df["y"] = pd.to_numeric(sales_df["y"], errors="coerce").fillna(0)

# ------------------------------------------------------------------------------
# 3. DARTS TIMESERIES CREATION
# ------------------------------------------------------------------------------
print("Building TimeSeries objects...")
series_list = TimeSeries.from_group_dataframe(
    sales_df,
    group_cols="unique_id",
    time_col="ds",
    value_cols="y",
    fill_missing_dates=True,
    freq='D'
)

# ------------------------------------------------------------------------------
# 4. COVARIATES GENERATION
# ------------------------------------------------------------------------------
print("Generating covariates...")

# We need covariates for History (d_1...d_1941) + Forecast Horizon (d_1942...d_1969)
min_date = sales_df['ds'].min()
max_date = sales_df['ds'].max()
forecast_end_date = max_date + timedelta(days=HORIZON)

full_time_index = pd.date_range(start=min_date, end=forecast_end_date, freq='D')

global_cov_ts = TimeSeries.from_times_and_values(
    times=full_time_index,
    values=np.zeros(len(full_time_index))
)

month_cov = datetime_attribute_timeseries(global_cov_ts, attribute="month", one_hot=True)
day_cov = datetime_attribute_timeseries(global_cov_ts, attribute="dayofweek", one_hot=True)
global_covariates = month_cov.stack(day_cov)

covariates_list = [global_covariates] * len(series_list)

# Scale targets
print("Scaling targets...")
scaler = Scaler()
series_list_scaled = scaler.fit_transform(series_list)

# ------------------------------------------------------------------------------
# 5. MODEL TRAINING
# ------------------------------------------------------------------------------
print("Initializing N-BEATS...")
model = NBEATSModel(
    input_chunk_length=int(input_chunk_length),
    output_chunk_length=int(output_chunk_length),
    n_epochs=int(n_epochs),
    random_state=42,
    model_name="nbeats-full-history",
    force_reset=True,
    save_checkpoints=False,
    pl_trainer_kwargs={
        "accelerator": "gpu" if torch.cuda.is_available() else "cpu",
        "devices": 1
    },
    batch_size=1024
)

print("Fitting model on combined (Train + Validation) history...")
model.fit(
    series=series_list_scaled,
    past_covariates=covariates_list,
    verbose=True
)

# ------------------------------------------------------------------------------
# 6. FORECASTING
# ------------------------------------------------------------------------------
print(f"Predicting next {HORIZON} days (d_1942 - d_1969)...")
preds_scaled = model.predict(
    n=HORIZON,
    series=series_list_scaled,
    past_covariates=covariates_list
)

print("Inverse scaling predictions...")
preds = scaler.inverse_transform(preds_scaled)

# ------------------------------------------------------------------------------
# 7. EVALUATION (RMSE vs TEST EVALUATION SET)
# ------------------------------------------------------------------------------
print("Calculating RMSE against Ground Truth (sales_test_evaluation_afcs_2025.csv)...")

# 1. Prepare Ground Truth Array
# Select only the relevant d_columns (d_1942 ... d_1969)
eval_cols = get_d_cols(eval_df)
# Ensure we sort the ground truth rows by ID to match the predictions order
# (We already aligned eval_df IDs in Step 2, but good to be safe)
y_true = eval_df[eval_cols].values

# 2. Prepare Prediction Array
# preds is a list of TimeSeries. We need to stack them into a matrix (N_series x Horizon)
pred_vals = []
for i, ts in enumerate(preds):
    pred_vals.append(ts.values().flatten())

y_pred = np.array(pred_vals)

# 3. Compute RMSE
# We compute one global RMSE (all series, all days)
mse = mean_squared_error(y_true, y_pred)
rmse = sqrt(mse)

print("="*40)
print(f"GLOBAL RMSE: {rmse:.5f}")
print("="*40)

# Optional: Save predictions to CSV for inspection
print("Saving predictions to 'submission_nbeats_eval.csv'...")
f_cols = [f"F{x}" for x in range(1, HORIZON + 1)]
submission_df = pd.DataFrame(y_pred, columns=f_cols)
submission_df.insert(0, 'id', ids) # Use the aligned IDs
submission_df.to_csv("submission_nbeats_eval.csv", index=False)
print("Done.")

CUDA Available: True
Device Name: NVIDIA A100-SXM4-80GB
Loading data...
Training cols: 1913, Validation cols: 28
Reshaping training data (Melt)... this may take a moment.
Converting dates...
Building TimeSeries objects...
Generating covariates...
Scaling targets...


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1551: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return _C._get_float32_matmul_precision()
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tenso

Initializing N-BEATS...
Fitting model on combined (Train + Validation) history...


INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 10.5 M | train
-------------------------------------------------------------
10.5 M    Trainable params
4.6 K     Non-trainable params
10.5 M    Total params
41.832    Total estimated model params size (MB)
396       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]